<a href="https://colab.research.google.com/github/ryaltic/Modeling-Ranked-Match-Outcomes-Using-Early-Game-Performance-Data-in-League-of-Legends/blob/main/Modeling_Ranked_Matches_Outcomes_LOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the data and doing Pre-processing

In [19]:
# Loading the pandas package
import pandas as pd

In [20]:
# Loading in the data file from my github
matches = pd.read_csv('https://raw.githubusercontent.com/ryaltic/Modeling-Ranked-Match-Outcomes-Using-Early-Game-Performance-Data-in-League-of-Legends/refs/heads/main/league_matches_data.csv')

In [21]:
# Displaying the first few rows
matches.head()

,match_id,minute,game_duration,blue_win,game_version,Blue_TOP_kills,Blue_TOP_gold,Blue_TOP_xp,Blue_TOP_cs,Blue_TOP_wards_placed,...,Blue__gold,Blue__xp,Blue__cs,Blue__wards_placed,Blue__wards_killed,Red__gold,Red__xp,Red__cs,Red__wards_placed,Red__wards_killed
0,5228482907,5,1666,1,15.3.656.4086,1,1698,2050,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5228482907,10,1666,1,15.3.656.4086,0,3262,4551,49,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5228482907,15,1666,1,15.3.656.4086,1,5214,6996,86,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5210847800,5,1848,1,15.1.649.4112,0,1643,1648,13,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5210847800,10,1848,1,15.1.649.4112,0,3236,4174,44,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Each match contains three data points and shows all information about the match at that current moment. There are a few columns that has information about the game not pertaining to that current moment in time which are game duration, the team that won, the game version and match id. The columns are named well but there will need to be some conversions that will need to be done.  

In [22]:
# Displaying the columns and column types
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80515 entries, 0 to 80514
Data columns (total 79 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   match_id                   80515 non-null  int64  
 1   minute                     80515 non-null  int64  
 2   game_duration              80515 non-null  int64  
 3   blue_win                   80515 non-null  int64  
 4   game_version               80515 non-null  object 
 5   Blue_TOP_kills             80515 non-null  int64  
 6   Blue_TOP_gold              80515 non-null  int64  
 7   Blue_TOP_xp                80515 non-null  int64  
 8   Blue_TOP_cs                80515 non-null  int64  
 9   Blue_TOP_wards_placed      80515 non-null  int64  
 10  Blue_TOP_wards_killed      80515 non-null  int64  
 11  Blue_JUNGLE_kills          80515 non-null  int64  
 12  Blue_JUNGLE_gold           80515 non-null  int64  
 13  Blue_JUNGLE_xp             80515 non-null  int

Most of the columns are numeric and there are a few columns that have a majority of the data missing which will be dropped. The columns that are going to be dropped are team totals that can be remade.

In [23]:
# Displaying the shape of the df
matches.shape

(80515, 79)

There are a total of 80,515 rows and 79 columns from the csv file. There is around 27,000 individual matches.

In [24]:
# Dropping the columns that were majority missing
matches = matches.drop(labels= ['Blue__gold', 'Blue__xp', 'Blue__cs', 'Blue__wards_placed', 'Blue__wards_killed',
                                'Red__gold', 'Red__xp', 'Red__cs', 'Red__wards_placed', 'Red__wards_killed'],
                        axis = 1)

<bound method DataFrame.info of          match_id  minute  game_duration  blue_win    game_version  \
0      5228482907       5           1666         1   15.3.656.4086   
1      5228482907      10           1666         1   15.3.656.4086   
2      5228482907      15           1666         1   15.3.656.4086   
3      5210847800       5           1848         1   15.1.649.4112   
4      5210847800      10           1848         1   15.1.649.4112   
...           ...     ...            ...       ...             ...   
80510  5348110836      10           1629         1  15.16.704.6097   
80511  5348110836      15           1629         1  15.16.704.6097   
80512  5347708304       5           1886         1  15.16.704.6097   
80513  5347708304      10           1886         1  15.16.704.6097   
80514  5347708304      15           1886         1  15.16.704.6097   

       Blue_TOP_kills  Blue_TOP_gold  Blue_TOP_xp  Blue_TOP_cs  \
0                   1           1698         2050           22   
1                   0           3262         4551           49   
2                   1           5214         6996           86   
3                   0           1643         1648           13   
4                   0           3236         4174           44   
...               ...            ...          ...          ...   
80510               1           3344         4576           53   
80511               0           5588         7688           96   
80512               0           1661         2125           37   
80513               0           3714         4994           74   
80514               0           5748         8212          128   

       Blue_TOP_wards_placed  ...  Red_UTILITY_kills  Red_UTILITY_gold  \
0                          0  ...                  0              1413   
1                          0  ...                  0              3116   
2                          0  ...                  0              4519   
3                          0  ...                  0              1304   
4                          0  ...                  0              2448   
...                      ...  ...                ...               ...   
80510                      0  ...                  0              2225   
80511                      0  ...                  0              3241   
80512                      0  ...                  0              1316   
80513                      0  ...                  0              2526   
80514                      0  ...                  0              3877   

       Red_UTILITY_xp  Red_UTILITY_cs  Red_UTILITY_wards_placed  \
0                1192               5                         0   
1                2928              10                         0   
2                4497              16                         0   
3                1165               8                         0   
4                2873              21                         0   
...               ...             ...                       ...   
80510            2166               9                         0   
80511            3515              13                         0   
80512            1138               8                         0   
80513            3108              18                         0   
80514            4654              31                         0   

       Red_UTILITY_wards_killed  blue_dragons  red_dragons  blue_grubs  \
0                             0             0            0           0   
1                             0             0            0           0   
2                             0             0            0           0   
3                             0             0            0           0   
4                             0             0            0           0   
...                         ...           ...          ...         ...   
80510                         0             0            0           0   
80511                         0             0   

In [29]:
# Only keeping rows that are at 10 minutes
matches_10 = matches[matches['minute'] == 10]
print(matches_10.head())
print(matches_10.shape)

      match_id  minute  game_duration  blue_win    game_version  \
1   5228482907      10           1666         1   15.3.656.4086   
4   5210847800      10           1848         1   15.1.649.4112   
7   5208847505      10           1484         1   15.1.649.4112   
10  5197140868      10           2007         1  14.24.644.2327   
13  5197107354      10           1857         1  14.24.644.2327   

    Blue_TOP_kills  Blue_TOP_gold  Blue_TOP_xp  Blue_TOP_cs  \
1                0           3262         4551           49   
4                0           3236         4174           44   
7                0           4085         5173           85   
10               0           2542         4280           36   
13               1           4177         4976           75   

    Blue_TOP_wards_placed  ...  Red_UTILITY_kills  Red_UTILITY_gold  \
1                       0  ...                  0              3116   
4                       0  ...                  0              2448   
7    

26,833 rows after filtering to matches at 10 minute time frame

In [30]:
# Checking for duplicate matches
matches_10.duplicated(subset = 'match_id').sum()

np.int64(0)

No duplicate matches

In [31]:
# Dropping more columns that are no longer needed
matches_10 = matches_10.drop(labels = ['minute','game_version','match_id','game_duration'], axis = 1)

Minute is no longer needed now that all games are 10 minutes, game version isn't need as the dataset doesn't have champions so the game version doesn't give any information. After checking for duplicated match_id is not needed anymore and game_duration was removed because it gave information about the game after the game was over.

## Splitting into Training and Testing Sets